## Load datasets

In [1]:
import pandas as pd

# files = {
#     'none': pd.read_csv('../deepseek-coder-6.7b-instruct/deepseek-coder-6.7b-instruct.csv'),
#     'cot' : pd.read_csv('../deepseek-coder-6.7b-instruct/CoT_deepseek-coder-6.7b-instruct.csv'),
#     'cot_ont' : pd.read_csv('../deepseek-coder-6.7b-instruct/CoT_ont_rag_deepseek-coder-6.7b-instruct.csv'),
#     'cot_ont_fs' : pd.read_csv('../deepseek-coder-6.7b-instruct/FS_CoT_ont_rag_deepseek-coder-6.7b-instruct.csv'),
#     'cot_fs' : pd.read_csv('../deepseek-coder-6.7b-instruct/FS_CoT_deepseek-coder-6.7b-instruct.csv'),
#     'ont' : pd.read_csv('../deepseek-coder-6.7b-instruct/ont_rag_deepseek-coder-6.7b-instruct.csv'),
#     'ont_fs' : pd.read_csv('../deepseek-coder-6.7b-instruct/FS_ont_rag_deepseek-coder-6.7b-instruct.csv'),
#     'fs' : pd.read_csv('../deepseek-coder-6.7b-instruct/FS_deepseek-coder-6.7b-instruct.csv'),
# }

gold_standard = pd.read_csv('../datafiles/lcquad_test_gold.csv')
gold_standard = gold_standard.sample(n=100, random_state=42)

In [2]:
#model_output = pd.read_csv('../datafiles/deepseek_output_21_01_2024.csv')
#model_output = pd.read_csv('../datafiles/codellama_output_22_01_2024.csv')
#model_output = pd.read_csv('../datafiles/deepseek_combined_fs8_schemalast_corrected_25_01_2024.csv')
#model_output = pd.read_csv('../datafiles/deepseek_combined_fs8_corrected_25_01_2024.csv')
#model_output =  pd.read_csv('../datafiles/deepseek_combined_corrected_23_01_2024.csv')
#model_output = pd.read_csv('../datafiles/gpt-3.5_30_01_2024.csv')
#model_output = pd.read_csv('../datafiles/deepseek_1k_02_02_2024.csv')
#model_output = pd.read_csv('../datafiles/CodeLlama_1k_14_02_2024.csv')
#model_output = pd.read_csv('../datafiles/gpt-3.5-turbo-0125_1k_25_2_2024.csv')
model_output = pd.read_csv('../results/AgentPosK5_codellama.csv')

In [3]:
model_output = model_output.fillna('none')

In [4]:
model_output['model'] = 'gpt-3.5'

## Preprocess

In [5]:
# String to python data structure
model_output['evaluated'] = model_output['output'].apply(eval)
gold_standard['evaluated'] = gold_standard['output'].apply(eval)

In [6]:
def preprocess(output):
    _result = set()
    if 'head' in output and 'results' in output:
        subset = set()
        for element in output['results']['bindings']:
            subset |= ({element[x]['value'] for x in element.keys()})
        _result = subset
    elif 'head' in output and 'boolean' in output:
        _result = output['boolean']
    else:
        print(output)
    return _result

In [7]:
def check_well_constructed(output):
    if 'output error' in output:
        return False
    return True

In [8]:
def type_of_consult(consult):
    result = 'NONE'
    if 'COUNT' in consult: result = 'COUNT'
    elif 'ASK' in consult: result = 'ASK'
    elif 'ORDER BY' in consult: result = 'ORDER_BY'
    elif 'SELECT' in consult: result = 'SELECT'
    return result

In [9]:
model_output['curated'] = model_output['evaluated'].apply(preprocess)

['output error: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. \n\nResponse:\nb"Virtuoso 37000 Error SP030: SPARQL compiler, line 6: syntax error at \'.\' before \'}\'\\n\\nSPARQL query:\\ndefine sql:big-data-const 0\\n#output-format:application/sparql-results+json\\n\\nSELECT DISTINCT?uri WHERE {\\n ?uri <http://dbpedia.org/ontology/nearestCity> <http://dbpedia.org/resource/Bend,_Oregon>.\\n ?uri <http://dbpedia.org/ontology/ProtectedArea>.\\n}\\n"']
['output error: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. \n\nResponse:\nb\'Virtuoso 37000 Error SP030: SPARQL compiler, line 14: syntax error at \\\'UNION\\\' before \\\'{\\\'\\n\\nSPARQL query:\\ndefine sql:big-data-const 0\\n#output-format:application/sparql-results+json\\n\\nPREFIX dbpedia: <http://dbpedia.org/ontology/>\\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\\n\\nSELECT DISTINCT?managerName\\nWHE

In [10]:
gold_standard['curated'] = gold_standard['evaluated'].apply(preprocess)

In [11]:
#model_output['method'] = 'all'

### Code Execution Eval

In [12]:
def compare_values(data1, data2):
    """Recursively compare values of two potentially complex data structures."""
    if type(data1) != type(data2):
        #print('type 1')
        return False
    
    if isinstance(data1, dict):
        if data1.keys() != data2.keys():
            #print("type 2")
            return False
        #print("type 3")
        return all(compare_values(data1[key], data2[key]) for key in data1)
    
    elif isinstance(data1, list):
        # Check if we need to care about the order
        if 'bindings' in str(data1) or 'bindings' in str(data2):
            #print("type 4")
            return all(compare_values(sub_data1, sub_data2) for sub_data1, sub_data2 in zip(data1, data2))
        else:
            #print('type 5')
            return sorted(data1) == sorted(data2)
    
    else:
        #print(type(data1), type(data2))
        return data1 == data2

In [13]:
def compare_values(data1, data2):
    """Recursively compare values of two potentially complex data structures without considering order."""
    if type(data1) != type(data2):
        return False
    
    if isinstance(data1, dict):
        if data1.keys() != data2.keys():
            return False
        return all(compare_values(data1[key], data2[key]) for key in data1)
    
    elif isinstance(data1, list):
        # Handle lists of dictionaries or other complex structures without considering order
        if len(data1) != len(data2):
            return False
        # Use a more complex method to ensure each element in one list is present in the other, regardless of order
        matched_elements = 0
        for item1 in data1:
            for item2 in data2:
                if compare_values(item1, item2):
                    matched_elements += 1
                    break
        return matched_elements == len(data1)
    
    else:
        return data1 == data2

In [14]:
grouped = model_output.groupby('method', sort=False)
ex_eval = pd.DataFrame()
for i, (method, group) in enumerate(grouped):
    comparison = group['curated'].values == gold_standard['curated'].values
    print(method, sum(comparison))
    #comparison = [compare_values(row1, row2) for row1, row2 in zip(group['curated'], gold_standard['curated'])]
    #print(f"Accuracy for {method}: {comparison.mean()}")
    comparison = pd.DataFrame(comparison, columns=['evaluation'])
    comparison['method'] = method
    comparison['consult'] = group['consult'].to_list()
    comparison['output'] = group['output'].to_list()
    print(sum(group['output'].apply(check_well_constructed)))
    ex_eval = pd.concat([ex_eval, comparison])

agent_generative 34
97


In [30]:
model_output['consult']

0     \nPREFIX dbpedia-owl: <http://dbpedia.org/onto...
1     \nSELECT DISTINCT?uri WHERE {\n ?uri <http://d...
2     \nSELECT DISTINCT?uri WHERE {\n  <http://dbped...
3     \nSELECT DISTINCT?uri WHERE {\n ?x <http://dbp...
4     \nSELECT DISTINCT COUNT(?uri) WHERE {\n ?uri <...
5     \nSELECT DISTINCT COUNT(?uri) WHERE {\n ?uri <...
6     \nSELECT DISTINCT?uri WHERE { <http://dbpedia....
7     \nASK WHERE { <http://dbpedia.org/resource/6-1...
8     \nSELECT DISTINCT?uri WHERE {\n ?uri <http://d...
9     \nSELECT DISTINCT?uri WHERE {\n ?uri <http://d...
10    \nSELECT DISTINCT?uri WHERE {\n ?x <http://dbp...
11    \nSELECT DISTINCT?associate WHERE {\n ?band <h...
12    \nSELECT DISTINCT COUNT(?uri) WHERE {\n ?x <ht...
13    \nSELECT DISTINCT?uri WHERE {\n ?uri <http://d...
Name: consult, dtype: object

In [27]:
gold_standard['corrected_question'].values[:14]

array(["What are Jerry Bock's musicals based upon?",
       'Name the company which produces both  Boeing F/A-18 E/F Hornet nad CH-46 Chinook ?',
       'What is the official language of Ladonia, which is the target area of Probus Journal?',
       'Name all the products produced by the producer of United States dollar ?',
       'How many bacterias are there whose division is Firmicutes?',
       'How many movies have been directed by Orson Welles ?',
       'Who won the silver medal when the bronze was won by Yang Yilin?',
       'Is George Lucas the cinematographer of 6-18-67?',
       'What is the alma mater of the scientist  whose PhD advisor is Jean-Claude Latombe ?',
       'Which city is known for the people working in the western penn hospital?',
       'Which region of Bannock is the origin od Spaghetti squash ?',
       'List the associates of bands which have a label by Motown ?',
       'How many home stadium are there, of the soccer club seasons whose chairman is Merritt 

In [53]:
ex_eval['evaluation'] = ex_eval['evaluation'].astype(int)

In [54]:
# for i in range(0,8000, 1000):
#     print(sum(ex_eval['evaluation'].to_list()[i:i+1000]))

In [55]:
# len(ex_eval[0].to_list())

In [56]:
# model_output['evaluation'] = ex_eval[0].to_list()
# model_output['evaluation'] = model_output['evaluation'].astype(int)

In [57]:
model_output.method

0               none
1               none
2               none
3               none
4               none
            ...     
7995    FSCoTont_rag
7996    FSCoTont_rag
7997    FSCoTont_rag
7998    FSCoTont_rag
7999    FSCoTont_rag
Name: method, Length: 8000, dtype: object

In [58]:
ex_eval.groupby('method').describe()

evaluation                                          
                  count   mean       std  min  25%  50%  75%  max
method                                                           
CoT              1000.0  0.146  0.353283  0.0  0.0  0.0  0.0  1.0
CoTont_rag       1000.0  0.098  0.297463  0.0  0.0  0.0  0.0  1.0
FS               1000.0  0.477  0.499721  0.0  0.0  0.0  1.0  1.0
FSCoT            1000.0  0.440  0.496635  0.0  0.0  0.0  1.0  1.0
FSCoTont_rag     1000.0  0.365  0.481671  0.0  0.0  0.0  1.0  1.0
FSont_rag        1000.0  0.343  0.474949  0.0  0.0  0.0  1.0  1.0
none             1000.0  0.007  0.083414  0.0  0.0  0.0  0.0  1.0
ont_rag          1000.0  0.005  0.070569  0.0  0.0  0.0  0.0  1.0

In [59]:
ex_eval[['method', 'evaluation']].groupby('method').describe()

evaluation                                          
                  count   mean       std  min  25%  50%  75%  max
method                                                           
CoT              1000.0  0.146  0.353283  0.0  0.0  0.0  0.0  1.0
CoTont_rag       1000.0  0.098  0.297463  0.0  0.0  0.0  0.0  1.0
FS               1000.0  0.477  0.499721  0.0  0.0  0.0  1.0  1.0
FSCoT            1000.0  0.440  0.496635  0.0  0.0  0.0  1.0  1.0
FSCoTont_rag     1000.0  0.365  0.481671  0.0  0.0  0.0  1.0  1.0
FSont_rag        1000.0  0.343  0.474949  0.0  0.0  0.0  1.0  1.0
none             1000.0  0.007  0.083414  0.0  0.0  0.0  0.0  1.0
ont_rag          1000.0  0.005  0.070569  0.0  0.0  0.0  0.0  1.0

In [60]:
ex_eval[['method','evaluation']].groupby('method').mean()

,evaluation
method,
CoT,0.146
CoTont_rag,0.098
FS,0.477
FSCoT,0.440
FSCoTont_rag,0.365
FSont_rag,0.343
none,0.007
ont_rag,0.005


In [61]:
ex_eval['type'] = gold_standard.sparql_query.apply(type_of_consult).to_list() * 8

In [62]:
ex_eval[['method','evaluation','type']].groupby(['method', 'type']).describe().round(4)

evaluation                                         
                         count    mean     std  min  25%  50%  75%  max
method       type                                                      
CoT          ASK          83.0  0.1807  0.3871  0.0  0.0  0.0  0.0  1.0
             COUNT       123.0  0.1057  0.3087  0.0  0.0  0.0  0.0  1.0
             SELECT      794.0  0.1486  0.3559  0.0  0.0  0.0  0.0  1.0
CoTont_rag   ASK          83.0  0.1084  0.3128  0.0  0.0  0.0  0.0  1.0
             COUNT       123.0  0.0488  0.2163  0.0  0.0  0.0  0.0  1.0
             SELECT      794.0  0.1045  0.3061  0.0  0.0  0.0  0.0  1.0
FS           ASK          83.0  0.5663  0.4986  0.0  0.0  1.0  1.0  1.0
             COUNT       123.0  0.4228  0.4960  0.0  0.0  0.0  1.0  1.0
             SELECT      794.0  0.4761  0.4997  0.0  0.0  0.0  1.0  1.0
FSCoT        ASK          83.0  0.5422  0.5012  0.0  0.0  1.0  1.0  1.0
             COUNT       123.0  0.3821  0.4879  0.0  0.0  0.0  1.0  1.0
             SELECT      794.0  0.4383  0.4965  0.0  0.0  0.0  1.0  1.0
FSCoTont_rag ASK          83.0  0.4819  0.5027  0.0  0.0  0.0  1.0  1.0
             COUNT       123.0  0.3089  0.4639  0.0  0.0  0.0  1.0  1.0
             SELECT      794.0  0.3615  0.4807  0.0  0.0  0.0  1.0  1.0
FSont_rag    ASK          83.0  0.5542  0.5001  0.0  0.0  1.0  1.0  1.0
             COUNT       123.0  0.3089  0.4639  0.0  0.0  0.0  1.0  1.0
             SELECT      794.0  0.3262  0.4691  0.0  0.0  0.0  1.0  1.0
none         ASK          83.0  0.0000  0.0000  0.0  0.0  0.0  0.0  0.0
             COUNT       123.0  0.0000  0.0000  0.0  0.0  0.0  0.0  0.0
             SELECT      794.0  0.0088  0.0935  0.0  0.0  0.0  0.0  1.0
ont_rag      ASK          83.0  0.0000  0.0000  0.0  0.0  0.0  0.0  0.0
             COUNT       123.0  0.0000  0.0000  0.0  0.0  0.0  0.0  0.0
             SELECT      794.0  0.0063  0.0792  0.0  0.0  0.0  0.0  1.0

In [63]:
ex_eval['model'] = 'CodeLlama-13b'

In [64]:
(ex_eval[['model', 'method','evaluation','type']].groupby(['model','method', 'type']).mean() * 100).round(2)

evaluation
model         method       type              
CodeLlama-13b CoT          ASK          18.07
                           COUNT        10.57
                           SELECT       14.86
              CoTont_rag   ASK          10.84
                           COUNT         4.88
                           SELECT       10.45
              FS           ASK          56.63
                           COUNT        42.28
                           SELECT       47.61
              FSCoT        ASK          54.22
                           COUNT        38.21
                           SELECT       43.83
              FSCoTont_rag ASK          48.19
                           COUNT        30.89
                           SELECT       36.15
              FSont_rag    ASK          55.42
                           COUNT        30.89
                           SELECT       32.62
              none         ASK           0.00
                           COUNT         0.00
                           SELECT        0.88
              ont_rag      ASK           0.00
                           COUNT         0.00
                           SELECT        0.63

In [65]:
output = ex_eval[['model', 'method','evaluation','type']].groupby(['model','method', 'type']).mean() * 100

In [66]:
print(output.to_latex())

\begin{tabular}{lllr}
\toprule
 &  &  & evaluation \\
model & method & type &  \\
\midrule
\multirow[t]{24}{*}{CodeLlama-13b} & \multirow[t]{3}{*}{CoT} & ASK & 18.072289 \\
 &  & COUNT & 10.569106 \\
 &  & SELECT & 14.861461 \\
\cline{2-4}
 & \multirow[t]{3}{*}{CoTont_rag} & ASK & 10.843373 \\
 &  & COUNT & 4.878049 \\
 &  & SELECT & 10.453401 \\
\cline{2-4}
 & \multirow[t]{3}{*}{FS} & ASK & 56.626506 \\
 &  & COUNT & 42.276423 \\
 &  & SELECT & 47.607053 \\
\cline{2-4}
 & \multirow[t]{3}{*}{FSCoT} & ASK & 54.216867 \\
 &  & COUNT & 38.211382 \\
 &  & SELECT & 43.828715 \\
\cline{2-4}
 & \multirow[t]{3}{*}{FSCoTont_rag} & ASK & 48.192771 \\
 &  & COUNT & 30.894309 \\
 &  & SELECT & 36.146096 \\
\cline{2-4}
 & \multirow[t]{3}{*}{FSont_rag} & ASK & 55.421687 \\
 &  & COUNT & 30.894309 \\
 &  & SELECT & 32.619647 \\
\cline{2-4}
 & \multirow[t]{3}{*}{none} & ASK & 0.000000 \\
 &  & COUNT & 0.000000 \\
 &  & SELECT & 0.881612 \\
\cline{2-4}
 & \multirow[t]{3}{*}{ont_rag} & ASK & 0.000000 \\

In [67]:
ex_eval['valid'] = ex_eval['output'].apply(check_well_constructed)

In [31]:
print(ex_eval[['method','valid']].groupby('method').mean().to_latex())

\begin{tabular}{lr}
\toprule
 & valid \\
method &  \\
\midrule
CoT & 0.896000 \\
CoTont_rag & 0.897000 \\
FS & 0.995000 \\
FSCoT & 0.972000 \\
FSCoTont_rag & 0.925000 \\
FSont_rag & 0.755000 \\
none & 0.266000 \\
ont_rag & 0.065000 \\
\bottomrule
\end{tabular}



In [35]:
ex_eval[ex_eval['method']=='ont_rag']['output'].tolist()

['[\'output error: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. \\n\\nResponse:\\nb"Virtuoso 37000 Error SP030: SPARQL compiler, line 6: Undefined namespace prefix at \\\'dbr\\\' before \\\':militaryCommand\\\'\\\\n\\\\nSPARQL query:\\\\ndefine sql:big-data-const 0\\\\n#output-format:application/sparql-results+json\\\\n\\\\nSELECT ?architect\\\\nWHERE {\\\\n  dbr:Marine_Corps_Air_Station_Kaneohe_Bay :militaryCommand ?architect .\\\\n  dbr:New_Sanno_Hotel :chain ?architect .\\\\n}\\\\n"\']',
 '[\'output error: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. \\n\\nResponse:\\nb"Virtuoso 37000 Error SP030: SPARQL compiler, line 6: Undefined namespace prefix at \\\'\\\' before \\\';\\\'\\\\n\\\\nSPARQL query:\\\\ndefine sql:big-data-const 0\\\\n#output-format:application/sparql-results+json\\\\n\\\\nSELECT ?sect ?politicalParty\\\\nWHERE {\\\\n  ?sect a :EthnicGroup ;\\\\n    